In [ ]:
import pyodbc
import pymysql
import time
from datetime import datetime, timedelta
import signal
import sys

# ================== تنظیمات ==================
INTERVAL_SECONDS = 60 * 60  # هر ۶۰ دقیقه یکبار اجرا
RUNNING = True

# ================== اتصالات ==================
def get_sql_connection():
    return pyodbc.connect(
        'DRIVER={SQL Server};'
        'SERVER=MKZ-DSAS\\DSAS;'
        'DATABASE=DSAS;'
        'UID=datadriven;'
        'PWD=5Rdx@4Rfv1355'
    )

def get_mysql_connection():
    return pymysql.connect(
        host="127.0.0.1",
        port=3306,
        user="root",
        password="",
        database="dsas",
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor
    )

# ================== تنظیمات جدول BLADE ==================
TABLE_NAME = "main_table_mhi_blade_11"
UNIT_ID = 11
MAIN_ASSET_ID = 9391

# AssetIDهای فرعی (از 9392 تا 9407)
SUB_ASSET_IDS = list(range(9392, 9408))  # 9392 تا 9407 → ۱۶ عدد

# نگاشت خودکار AssetID → نام فیلد
field_map = {aid: f"AssetID_{aid}" for aid in [MAIN_ASSET_ID] + SUB_ASSET_IDS}

# ================== تابع اصلی ==================
def run_task():
    print(f"\n{'='*100}")
    print(f" شروع همگام‌سازی BLADE | زمان: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f" جدول هدف: {TABLE_NAME}")
    print(f" unitID: {UNIT_ID} | AssetID اصلی: {MAIN_ASSET_ID}")
    print(f" تعداد فیلدهای فرعی: {len(SUB_ASSET_IDS)} (9392 تا 9407)")
    print(f"{'='*100}")

    conn_sql = None
    conn_mysql = None
    cursor_sql = None

    try:
        print("اتصال به SQL Server...")
        conn_sql = get_sql_connection()
        cursor_sql = conn_sql.cursor()

        print("اتصال به MySQL...")
        conn_mysql = get_mysql_connection()

        # دریافت TimeStampsهای موجود
        print("در حال بررسی رکوردهای موجود در جدول MySQL...")
        existing_ts = set()
        with conn_mysql.cursor() as c:
            c.execute(f"SELECT TimeStamps FROM {TABLE_NAME} WHERE unitID = %s", (UNIT_ID,))
            for row in c.fetchall():
                if row['TimeStamps'] is not None:
                    existing_ts.add(row['TimeStamps'])
        print(f"تعداد رکورد موجود در جدول: {len(existing_ts):,} رکورد")

        # دریافت آخرین ۱۲۰۰۰ رکورد از AssetID اصلی (9391)
        print(f"در حال دریافت آخرین ۱۲۰۰۰ رکورد از AssetID={MAIN_ASSET_ID}...")
        query_main = """
            SELECT TOP 12000 [Value], [RecordTime], [RecordDate], [DateTime], [TimeStamp]
            FROM [DSAS].[PDA].[Periodic_Values]
            WHERE [UnitID] = ? AND [AssetID] = ?
            ORDER BY [DateTime] DESC
        """
        cursor_sql.execute(query_main, (UNIT_ID, MAIN_ASSET_ID))
        rows_main = cursor_sql.fetchall()

        if not rows_main:
            print(f"هیچ داده‌ای برای AssetID={MAIN_ASSET_ID} یافت نشد.")
            return

        inserted_count = 0
        for idx, row in enumerate(rows_main):
            ts = row.TimeStamp
            if ts in existing_ts:
                continue  # تکراری → رد شود (بدون پیام)

            dt = row.DateTime
            main_value = float(row.Value) if row.Value is not None else None

            print(f"\n{'─'*80}")
            print(f"درج رکورد جدید {idx+1:,} | TimeStamps: {ts} | {dt.strftime('%Y-%m-%d %H:%M:%S')}")

            # ۱. درج رکورد اصلی (فقط AssetID_9391)
            insert_sql = f"""
                INSERT INTO {TABLE_NAME} 
                (unitID, AssetID_9391, RecordTime, RecordDate, DateTime, TimeStamps)
                VALUES (%s, %s, %s, %s, %s, %s)
            """
            try:
                with conn_mysql.cursor() as c:
                    c.execute(insert_sql, (
                        UNIT_ID,
                        main_value,
                        dt.strftime('%H:%M:%S'),
                        dt.strftime('%Y-%m-%d'),
                        dt,
                        ts
                    ))
                conn_mysql.commit()
                inserted_count += 1
                print("رکورد اصلی با موفقیت درج شد.")
            except Exception as e:
                print(f"خطا در درج رکورد اصلی: {e}")
                conn_mysql.rollback()
                continue

            # ۲. پر کردن ۱۶ فیلد فرعی
            for aid in SUB_ASSET_IDS:
                field_name = field_map[aid]

                # جستجوی نزدیک (اختلاف ≤ ۱۰۰۰)
                near_sql = """
                    SELECT TOP 1 [Value]
                    FROM [DSAS].[PDA].[Periodic_Values]
                    WHERE [UnitID]=? AND [AssetID]=? AND ABS([TimeStamp] - ?) <= 1000
                    ORDER BY ABS([TimeStamp] - ?)
                """
                cursor_sql.execute(near_sql, (UNIT_ID, aid, ts, ts))
                near_row = cursor_sql.fetchone()

                if near_row and near_row.Value is not None:
                    value = float(near_row.Value)
                    source = "نزدیک"
                else:
                    # میانگین ۱۰ مقدار آخر
                    avg_sql = """
                        SELECT AVG(CAST([Value] AS FLOAT))
                        FROM (SELECT TOP 10 [Value]
                              FROM [DSAS].[PDA].[Periodic_Values]
                              WHERE [UnitID]=? AND [AssetID]=?
                              ORDER BY [DateTime] DESC) sub
                    """
                    cursor_sql.execute(avg_sql, (UNIT_ID, aid))
                    avg_row = cursor_sql.fetchone()
                    value = round(float(avg_row[0]), 4) if avg_row and avg_row[0] is not None else None
                    source = "میانگین ۱۰ تایی"

                if value is not None:
                    update_sql = f"UPDATE {TABLE_NAME} SET {field_name} = %s WHERE TimeStamps = %s"
                    try:
                        with conn_mysql.cursor() as c:
                            c.execute(update_sql, (value, ts))
                        conn_mysql.commit()
                        print(f"   {field_name} = {value} ({source})")
                    except Exception as e:
                        print(f"   خطا در آپدیت {field_name}: {e}")
                        conn_mysql.rollback()
                else:
                    print(f"   {field_name}: مقدار یافت نشد.")

        print(f"\n{'='*100}")
        print(f"خلاصه این اجرا: {inserted_count:,} رکورد جدید با موفقیت درج و تکمیل شد.")
        print(f"{'='*100}")

    except Exception as e:
        print(f"خطای کلی در اجرا: {e}")
    finally:
        if cursor_sql: cursor_sql.close()
        if conn_sql: conn_sql.close()
        if conn_mysql: conn_mysql.close()

# ================== توقف با Ctrl+C ==================
def signal_handler(sig, frame):
    global RUNNING
    print('\nدریافت سیگنال توقف (Ctrl+C) → در حال خروج...')
    RUNNING = False
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

# ================== حلقه زمان‌بندی ==================
def scheduler():
    print("اسکریپت همگام‌سازی BLADE شروع شد.")
    print("هر ۶۰ دقیقه یکبار اجرا می‌شود. برای توقف Ctrl+C بزنید.\n")
    while RUNNING:
        run_task()
        if RUNNING:
            next_run = datetime.now() + timedelta(seconds=INTERVAL_SECONDS)
            print(f"\nمنتظر اجرای بعدی: {next_run.strftime('%Y-%m-%d %H:%M:%S')} ...")
            for _ in range(INTERVAL_SECONDS):
                if not RUNNING:
                    break
                time.sleep(1)

if __name__ == "__main__":
    scheduler()

اسکریپت همگام‌سازی BLADE شروع شد.
هر ۶۰ دقیقه یکبار اجرا می‌شود. برای توقف Ctrl+C بزنید.


 شروع همگام‌سازی BLADE | زمان: 2025-11-20 10:05:19
 جدول هدف: main_table_mhi_blade_11
 unitID: 11 | AssetID اصلی: 9391
 تعداد فیلدهای فرعی: 16 (9392 تا 9407)
اتصال به SQL Server...
اتصال به MySQL...
در حال بررسی رکوردهای موجود در جدول MySQL...
تعداد رکورد موجود در جدول: 0 رکورد
در حال دریافت آخرین ۱۲۰۰۰ رکورد از AssetID=9391...

────────────────────────────────────────────────────────────────────────────────
درج رکورد جدید 1 | TimeStamps: 1687862682 | 2023-06-27 14:14:42
رکورد اصلی با موفقیت درج شد.
   AssetID_9392 = -1.0 (نزدیک)
   AssetID_9393 = -9.0 (نزدیک)
   AssetID_9394 = 40.0 (نزدیک)
   AssetID_9395 = 3.0 (نزدیک)
   AssetID_9396 = -2.0 (نزدیک)
   AssetID_9397 = 9.0 (نزدیک)
   AssetID_9398 = 1.0 (نزدیک)
   AssetID_9399 = 16.0 (نزدیک)
   AssetID_9400 = -10.0 (نزدیک)
   AssetID_9401 = -1.0 (نزدیک)
   AssetID_9402 = -8.0 (نزدیک)
   AssetID_9403 = -22.0 (نزدیک)
   AssetID_9404 = -10.0 (نزدیک)
 